In [1]:
import pyAgrum as gum
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import os
import itertools

from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression, LogisticRegression
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.preprocessing import PolynomialFeatures
pd.set_option('display.max_rows', 200)

In [2]:
df = pd.read_csv("outputs/scores2.csv", index_col=0)
df

,items,nodes,timesteps,arcs_ratio,islands,values,algorithm,scorefunction,recall,precision,fscore,dist2opt,tp,tn,fp,fn,nodestates,arcs,items/nodestates,items/arcs
0,1000,4,1,0.999999,1,2,3OFF2,AIC,0.666667,1.000000,0.800000,0.333333,2,9,0,1,16,3,62.500000,333.333333
1,1000,4,1,0.999999,1,2,3OFF2,AIC,0.000000,0.000000,0.000000,1.414214,0,7,2,3,16,3,62.500000,333.333333
2,1000,4,1,0.999999,1,2,3OFF2,AIC,0.000000,0.000000,0.000000,1.414214,0,7,2,3,16,3,62.500000,333.333333
3,1000,4,1,0.999999,1,2,3OFF2,AIC,0.333333,0.500000,0.400000,0.833333,1,8,1,2,16,3,62.500000,333.333333
4,1000,4,1,0.999999,1,2,3OFF2,AIC,0.000000,0.000000,0.000000,1.414214,0,7,2,3,16,3,62.500000,333.333333
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
129785,10000,8,3,2.000000,1,4,TABU,K2,0.527273,0.467742,0.495726,0.711878,29,464,33,26,65536,16,0.152588,625.000000
129786,10000,8,3,2.000000,1,4,TABU,K2,0.672727,0.521127,0.587302,0.580023,37,463,34,18,65536,16,0.152588,625.000000
129787,10000,8,3,2.000000,1,4,TABU,K2,0.890909,0.777778,0.830508,0.247555,49,483,14,6,65536,16,0.152588,625.000000
129788,10000,8,3,2.000000,1,4,TABU,K2,1.000000,0.785714,0.880000,0.214286,55,482,15,0,65536,16,0.152588,625.000000


In [3]:
def normalize(col):
    # TODO: GROTER DAN WAT????
    if max(col) > 1:
        return col/max(col)
    return col

columns = ['items', 'nodes', 'timesteps', 'arcs_ratio', 'islands', 'values', 'nodestates', 'arcs', 'items/nodestates', 'items/arcs']
df2 = df.copy()
df2[columns] = df2[columns].apply(normalize)
df2

,items,nodes,timesteps,arcs_ratio,islands,values,algorithm,scorefunction,recall,precision,fscore,dist2opt,tp,tn,fp,fn,nodestates,arcs,items/nodestates,items/arcs
0,0.1,0.5,0.333333,0.499999,1,0.5,3OFF2,AIC,0.666667,1.000000,0.800000,0.333333,2,9,0,1,0.000244,0.1875,0.100000,0.1000
1,0.1,0.5,0.333333,0.499999,1,0.5,3OFF2,AIC,0.000000,0.000000,0.000000,1.414214,0,7,2,3,0.000244,0.1875,0.100000,0.1000
2,0.1,0.5,0.333333,0.499999,1,0.5,3OFF2,AIC,0.000000,0.000000,0.000000,1.414214,0,7,2,3,0.000244,0.1875,0.100000,0.1000
3,0.1,0.5,0.333333,0.499999,1,0.5,3OFF2,AIC,0.333333,0.500000,0.400000,0.833333,1,8,1,2,0.000244,0.1875,0.100000,0.1000
4,0.1,0.5,0.333333,0.499999,1,0.5,3OFF2,AIC,0.000000,0.000000,0.000000,1.414214,0,7,2,3,0.000244,0.1875,0.100000,0.1000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
129785,1.0,1.0,1.000000,1.000000,1,1.0,TABU,K2,0.527273,0.467742,0.495726,0.711878,29,464,33,26,1.000000,1.0000,0.000244,0.1875
129786,1.0,1.0,1.000000,1.000000,1,1.0,TABU,K2,0.672727,0.521127,0.587302,0.580023,37,463,34,18,1.000000,1.0000,0.000244,0.1875
129787,1.0,1.0,1.000000,1.000000,1,1.0,TABU,K2,0.890909,0.777778,0.830508,0.247555,49,483,14,6,1.000000,1.0000,0.000244,0.1875
129788,1.0,1.0,1.000000,1.000000,1,1.0,TABU,K2,1.000000,0.785714,0.880000,0.214286,55,482,15,0,1.000000,1.0000,0.000244,0.1875


# Linear Regression

In [4]:
columns = ['items', 'nodes', 'timesteps', 'arcs_ratio', 'islands', 'values', 'nodestates', 'arcs', 'items/nodestates', 'items/arcs']

# Splitting the data into training and testing data
X_train, X_test, y_train, y_test = train_test_split(df2[columns], df2["fscore"], test_size = 0.2)

poly = PolynomialFeatures(degree=2)
X_train_poly, X_test_poly = poly.fit_transform(X_train), poly.fit_transform(X_test)

regr = LinearRegression()

regr.fit(X_train_poly, y_train)

regr.score(X_test_poly, y_test)

0.2135825586515584

In [5]:
columns = ['items', 'nodes', 'timesteps', 'arcs_ratio', 'islands', 'values', 'nodestates', 'arcs', 'items/nodestates', 'items/arcs']

# Splitting the data into training and testing data
X_train, X_test, y_train, y_test = train_test_split(df2[columns], df2["fscore"], test_size = 0.2)

poly = PolynomialFeatures(degree=3)
X_train_poly, X_test_poly = poly.fit_transform(X_train), poly.fit_transform(X_test)

regr = LinearRegression()

regr.fit(X_train_poly, y_train)

regr.score(X_test_poly, y_test)

0.22571144147313138

In [6]:
columns = ['items', 'nodes', 'timesteps', 'arcs_ratio', 'islands', 'values', 'nodestates', 'arcs', 'items/nodestates', 'items/arcs']

# Splitting the data into training and testing data
X_train, X_test, y_train, y_test = train_test_split(df2[columns], df2["fscore"], test_size = 0.2)

poly = PolynomialFeatures(degree=4)
X_train_poly, X_test_poly = poly.fit_transform(X_train), poly.fit_transform(X_test)

regr = LinearRegression()

regr.fit(X_train_poly, y_train)

regr.score(X_test_poly, y_test)

0.21526794303789543

# Logistic Regression

In [7]:
dfbin = df2.copy()
dfbin["fscore"] = pd.cut(dfbin["fscore"],bins=[-0.01,0.5,1.0], labels=['0','1'])

columns = ['items', 'nodes', 'timesteps', 'arcs_ratio', 'islands', 'values', 'nodestates', 'arcs', 'items/nodestates', 'items/arcs']

# Splitting the data into training and testing data
X_train, X_test, y_train, y_test = train_test_split(dfbin[columns], dfbin["fscore"], test_size = 0.2)

poly = PolynomialFeatures(degree=2)
X_train_poly, X_test_poly = poly.fit_transform(X_train), poly.fit_transform(X_test)

X_test_poly

regr = LogisticRegression(max_iter=100)

regr.fit(X_train_poly, y_train)

regr.score(X_test_poly, y_test)

C:\Users\jacco\anaconda3\envs\thesis\lib\site-packages\sklearn\linear_model\_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


0.7066800215733108

In [8]:
dfbin = df2.copy()
dfbin["fscore"] = pd.cut(dfbin["fscore"],bins=[-0.01,0.33,0.66,1.0], labels=['0','1','2'])

columns = ['items', 'nodes', 'timesteps', 'arcs_ratio', 'islands', 'values', 'nodestates', 'arcs', 'items/nodestates', 'items/arcs']

# Splitting the data into training and testing data
X_train, X_test, y_train, y_test = train_test_split(dfbin[columns], dfbin["fscore"], test_size = 0.2)

poly = PolynomialFeatures(degree=2)
X_train_poly, X_test_poly = poly.fit_transform(X_train), poly.fit_transform(X_test)

X_test_poly

regr = LogisticRegression(max_iter=100)

regr.fit(X_train_poly, y_train)

regr.score(X_test_poly, y_test)

C:\Users\jacco\anaconda3\envs\thesis\lib\site-packages\sklearn\linear_model\_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


0.5678018337314122

In [9]:
dfbin = df2.copy()
dfbin["fscore"] = pd.cut(dfbin["fscore"],bins=[-0.01,0.25,0.5,0.75,1.0], labels=['0','1','2','3'])

columns = ['items', 'nodes', 'timesteps', 'arcs_ratio', 'islands', 'values', 'nodestates', 'arcs', 'items/nodestates', 'items/arcs']

# Splitting the data into training and testing data
X_train, X_test, y_train, y_test = train_test_split(dfbin[columns], dfbin["fscore"], test_size = 0.2)

poly = PolynomialFeatures(degree=2)
X_train_poly, X_test_poly = poly.fit_transform(X_train), poly.fit_transform(X_test)

X_test_poly

regr = LogisticRegression(max_iter=100)

regr.fit(X_train_poly, y_train)

regr.score(X_test_poly, y_test)

C:\Users\jacco\anaconda3\envs\thesis\lib\site-packages\sklearn\linear_model\_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


0.48347330302796826

In [10]:
dfbin = df2.copy()
dfbin["fscore"] = pd.cut(dfbin["fscore"],bins=[-0.01,0.2,0.4,0.6,0.8,1.0], labels=['0','1','2','3','4'])

columns = ['items', 'nodes', 'timesteps', 'arcs_ratio', 'islands', 'values', 'nodestates', 'arcs', 'items/nodestates', 'items/arcs']

# Splitting the data into training and testing data
X_train, X_test, y_train, y_test = train_test_split(dfbin[columns], dfbin["fscore"], test_size = 0.2)

poly = PolynomialFeatures(degree=2)
X_train_poly, X_test_poly = poly.fit_transform(X_train), poly.fit_transform(X_test)

X_test_poly

regr = LogisticRegression(max_iter=100)

regr.fit(X_train_poly, y_train)

regr.score(X_test_poly, y_test)

C:\Users\jacco\anaconda3\envs\thesis\lib\site-packages\sklearn\linear_model\_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


0.40808228677093766

In [11]:
dfbin = df2.copy()
dfbin["fscore"] = pd.cut(dfbin["fscore"],bins=[-0.01,0.1,0.2,0.3,0.4,0.5,0.6,0.7,0.8,0.9,1.0], labels=['0','1','2','3','4','5','6','7','8','9'])

columns = ['items', 'nodes', 'timesteps', 'arcs_ratio', 'islands', 'values', 'nodestates', 'arcs', 'items/nodestates', 'items/arcs']

# Splitting the data into training and testing data
X_train, X_test, y_train, y_test = train_test_split(dfbin[columns], dfbin["fscore"], test_size = 0.2)

poly = PolynomialFeatures(degree=2)
X_train_poly, X_test_poly = poly.fit_transform(X_train), poly.fit_transform(X_test)

X_test_poly

regr = LogisticRegression(max_iter=100)

regr.fit(X_train_poly, y_train)

regr.score(X_test_poly, y_test)

C:\Users\jacco\anaconda3\envs\thesis\lib\site-packages\sklearn\linear_model\_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


0.2741351413822328

# GradientBoosting

In [13]:
columns = ['items', 'nodes', 'timesteps', 'arcs_ratio', 'islands', 'values', 'nodestates', 'arcs', 'items/nodestates', 'items/arcs']

# Splitting the data into training and testing data
X_train, X_test, y_train, y_test = train_test_split(df2[columns], df2["fscore"], test_size = 0.2)

poly = PolynomialFeatures(degree=2)
X_train_poly, X_test_poly = poly.fit_transform(X_train), poly.fit_transform(X_test)

regr = GradientBoostingRegressor()

regr.fit(X_train_poly, y_train)

regr.score(X_test_poly, y_test)

0.22584879050728057